In [15]:
import numpy as np
# input_file = 'input_example.txt'
input_file = 'input_full.txt'

with open(input_file) as file:
    lines = file.readlines()
lines = [line.strip() for line in lines]
lines = [[int(x) for x in line.split(',')] for line in lines]
cubes = np.array(lines)
cubes

array([[ 4,  6, 13],
       [14,  2,  9],
       [ 4, 13,  6],
       ...,
       [ 5, 14, 15],
       [20,  8, 11],
       [ 7, 14, 19]])

In [16]:
# Part 1
moves = [np.array([1,0,0]), np.array([-1,0,0]), np.array([0,1,0]), np.array([0,-1,0]), np.array([0,0,1]), np.array([0,0,-1])]
nb_free = 0
for cube in cubes:
     for coord in [cube + move for move in moves]:
          if not(np.equal(cubes,coord).all(1).any()):
               nb_free += 1
print(f'Part 1: {nb_free}')

4288

In [17]:
maxs, mins = cubes.max(axis=0), cubes.min(axis=0)
maxx, maxy, maxz = maxs[0], maxs[1], maxs[2]
minx, miny, minz = mins[0], mins[1], mins[2]

def find_empty_cube():
    global cubes
    global empty_space
    for x in range(minx, maxx+1):
        for y in range(miny, maxy+1):
            for z in range(minz, maxz+1):
                coord = np.array([x, y, z])
                if not(np.equal(cubes, coord).all(1).any()):
                    if len(empty_space) == 0:
                        return coord
                    else:
                        if not(np.equal(empty_space, coord).all(1).any()):
                            return coord
    return np.array([])

def find_empty_neighbours(coord):
    # global cubes
    adjacent_coords = []
    for adjacent_coord in [coord + move for move in moves]:
        if (minx <= adjacent_coord[0] <= maxx) and (miny <= adjacent_coord[1] <= maxy) and (minz <= adjacent_coord[2] <= maxz):
            if not(np.equal(cubes,adjacent_coord).all(1).any()):
                adjacent_coords += [adjacent_coord]
    return adjacent_coords

def count_cube_connections(sub_empty_space):
    global cubes
    cube_connections = []
    for coord in sub_empty_space:
        for adjacent_coord in [coord + move for move in moves]:
            if np.equal(cubes,adjacent_coord).all(1).any():
                cube_connections += [adjacent_coord]
    return len(cube_connections)

empty_space = []
nb_to_remove = 0

while True:
    # Identify an empty space by starting with a remaining empty cube (i.e. not analyzed already)
    empty_cube = find_empty_cube()
    if len(empty_cube) == 0:
        break # In this case, there is no more empty cubes to analyze
    sub_empty_space = []
    empty_cubes_i = [empty_cube]
    while True:
        empty_space += empty_cubes_i
        sub_empty_space += empty_cubes_i
        empty_cube_i_plus_1 = []
        for empty_cube in empty_cubes_i:
            empty_neighbours = find_empty_neighbours(empty_cube)
            for empty_neighbour in empty_neighbours:
                if not(np.equal(empty_space, empty_neighbour).all(1).any()):
                    if len(empty_cube_i_plus_1) == 0 :
                        empty_cube_i_plus_1 += [empty_neighbour]
                    else:
                        if not(np.equal(empty_cube_i_plus_1, empty_neighbour).all(1).any()):
                            empty_cube_i_plus_1 += [empty_neighbour]
        if len(empty_cube_i_plus_1) == 0:
            break
        empty_cubes_i = empty_cube_i_plus_1.copy()
    # Analysis of the identified empty space (is it inside or outside?)
    maxs_i, mins_i = np.array(sub_empty_space).max(axis=0), np.array(sub_empty_space).min(axis=0)
    maxx_i, maxy_i, maxz_i = maxs_i[0], maxs_i[1], maxs_i[2]
    minx_i, miny_i, minz_i = mins_i[0], mins_i[1], mins_i[2]
    if not((minx_i == minx) or (miny_i == miny) or (minz_i == minz) or (maxx_i == maxx) or (maxy_i == maxy) or (maxz_i == maxz)):
        nb_to_remove_j = count_cube_connections(sub_empty_space)
        nb_to_remove += nb_to_remove_j

print(f'Part 2: {nb_free - nb_to_remove}')

2494
